In [1]:
from pyspark.sql import SparkSession

In [2]:
MAX_MEMORY = "5g"
spark = SparkSession.builder.appName("taxi-analysis")\
    .config("spark.executer.memory", MAX_MEMORY)\
    .config("spark.drive.memory", MAX_MEMORY).getOrCreate()

23/02/08 17:04:36 WARN Utils: Your hostname, imhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.89 instead (on interface en0)
23/02/08 17:04:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/08 17:04:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
trip_files = "/Users/imhaneul/Documents/sky-laboratory/spark-distribute/data/trips/*"

In [4]:
trips_df = spark.read.parquet(f"file:///{trip_files}")

In [6]:
trips_df.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [7]:
trips_df.createOrReplaceTempView("trips")

In [12]:
qs = """
SELECT 
    trip_distance,
    total_amount
FROM 
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 4
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""
spark.sql(qs).show()

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         16.5|       70.07|
|         1.13|       11.16|
|         2.68|       18.59|
|         12.4|        43.8|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
|         16.2|        45.3|
|         3.58|        19.3|
|         0.91|        14.8|
|         2.57|        12.8|
|          0.4|         5.3|
|         3.26|        17.3|
|        13.41|       47.25|
|         18.3|       61.42|
|         1.53|       14.16|
|          2.0|        11.8|
|         16.6|       54.96|
|         15.5|       56.25|
|          1.3|        16.8|
+-------------+------------+
only showing top 20 rows



In [13]:
data_df = spark.sql(qs)
data_df.createOrReplaceTempView("data")

In [14]:
data_df.describe().show()

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|          13083045|          13083045|
|   mean|2.8808531003293565|17.968543990358548|
| stddev| 3.819707118044885|12.972389388491917|
|    min|              0.01|              0.01|
|    max|             475.5|            4973.3|
+-------+------------------+------------------+



In [15]:
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=2023)

In [18]:
print(train_df.count())
print(test_df.count())

10466683


2616362


In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
vassembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")

In [21]:
vtrain_df = vassembler.transform(train_df)

In [23]:
vtrain_df.show()

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|        3.05|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 20 rows



In [24]:
from pyspark.ml.regression import LinearRegression

In [25]:
lr = LinearRegression(
    maxIter=50,
    regParam=0.01,
    labelCol="total_amount",
    featuresCol="features",
)

In [26]:
model = lr.fit(vtrain_df)

23/02/08 17:18:58 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/02/08 17:18:58 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/02/08 17:19:02 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [27]:
vtest_df = vassembler.transform(test_df)

In [30]:
pred = model.transform(vtest_df)

In [32]:
pred.show()

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.3|  [0.01]|9.463508950720904|
|         0.01|         3.8|  [0.01]|9.463508950720904|
|         0.01|         3.8|  [0.01]|9.463508950

In [35]:
model.summary.rootMeanSquaredError

6.141928718525298

In [36]:
model.summary.r2

0.7736562540300514

In [37]:
from pyspark.sql.types import DoubleType

distance = [1.1, 5.5, 30.0]
distance_df = spark.createDataFrame(distance, DoubleType()).toDF("trip_distance")

In [38]:
distance_df.show()

+-------------+
|trip_distance|
+-------------+
|          1.1|
|          5.5|
|         30.0|
+-------------+



In [39]:
vdistance_df = vassembler.transform(distance_df)

In [42]:
model.transform(vdistance_df).show()

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]|12.692563794059803|
|          5.5|   [5.5]|25.727280592859024|
|         30.0|  [30.0]| 98.30695367708196|
+-------------+--------+------------------+

